In [ ]:
import pandas as pd

benin_data = pd.read_csv('../data/benin-malanville.csv')
sierraleone_data = pd.read_csv('../data/sierraleone-bumbuna.csv')
togo_data = pd.read_csv('../data/togo-dapaong_qc.csv')

# Function to summarize data using pandas' describe() method, which returns summary statistics for all numeric columns by default.
def summarize_data(df):
    return df.describe(include='all')  # 'include=all' to get statistics for all columns

benin_summary = summarize_data(benin_data)
sierraleone_summary = summarize_data(sierraleone_data)
togo_summary = summarize_data(togo_data)

print("Benin Summary Statistics:\n", benin_summary)
print("Sierra Leone Summary Statistics:\n", sierraleone_summary)
print("Togo Summary Statistics:\n", togo_summary)

# Function for data qulity check using pandas isnull() abd select_dtypes() method.
def data_quality_check(df):
    missing_values = df.isnull().sum()
    negative_values = df[df.select_dtypes(include='number') < 0].count()
    return missing_values, negative_values

benin_missing, benin_negatives = data_quality_check(benin_data)
sierraleone_missing, sierraleone_negatives = data_quality_check(sierraleone_data)
togo_missing, togo_negatives = data_quality_check(togo_data)

print("Benin Data Quality Check:\n", benin_missing, benin_negatives)
print("Sierra Leone Data Quality Check:\n", sierraleone_missing, sierraleone_negatives)
print("Togo Data Quality Check:\n", togo_missing, togo_negatives)
